This notebook plots the solutions. You can actually plot the progress while the solver is running to see if it makes sense.

Various imports:

In [ ]:
%load_ext autoreload
%autoreload 2
# add path to code
import sys
sys.path.insert(0, '../source')
sys.path.insert(0, '../scripts')
import numpy as np
from matplotlib.tri import Triangulation
import matplotlib.pyplot as plt
import matplotlib as mpl
from pathlib import Path
import importlib
from dofs_to_serial import dofs_to_serial

In [ ]:
# set the results to load
resultsname = '../results/example'

In [ ]:
# load the setup module for that experiment
sys.path.insert(0, resultsname)
setup_name = Path(f'{resultsname}/model_info.txt').read_text()
setup = importlib.import_module(setup_name)

# create triangulation for matplotlib plotting that mirrors the mesh triangulation.
triang = Triangulation(setup.domain.geometry.x[:,0]/1e3,setup.domain.geometry.x[:,1]/1e3,setup.domain.geometry.dofmap)

In [ ]:
t = np.load(resultsname+'/t.npy')
nodes_x = np.load(resultsname+'/nodes_x.npy')
nodes_y = np.load(resultsname+'/nodes_y.npy')

# create map from parallel dof numbering to serial dof numbering
nodes_parallel = np.column_stack((nodes_x,nodes_y))
nodes_serial = setup.domain.geometry.x[:,0:2]
map_dofs = dofs_to_serial(nodes_parallel,nodes_serial)

In [ ]:
# load results and reorder dof's to the serial ordering
N = np.load(resultsname+'/N.npy')[:,map_dofs]
b = np.load(resultsname+'/b.npy')[:,map_dofs]
qx = np.load(resultsname+'/qx.npy')[:,map_dofs]
qy = np.load(resultsname+'/qy.npy')[:,map_dofs]
storage = np.load(resultsname+'/storage.npy')[:,map_dofs]

# get water flow speed
q_m = np.sqrt(qx**2+qy**2)

In [ ]:
# get mean of N over the lake and max of q not over the lake
N_mean = np.zeros(t.size)
q_max = np.zeros(t.size)

j_max=0

for j in range(t.size):
    if np.max(q_m[j,:]>1e-7):
        N_j = N[j,:]
        N_mean[j] = np.mean(N_j[storage[j,:]>1e-2])

        q_j = q_m[j,:]
        q_max[j] = np.max(q_j[storage[j,:]<1e-2])
        j_max+=1 
j_max -= 1

In [ ]:
# choose to plot a single timestep OR a sequence to make a movie
ind = [j_max]               # plot a single time step
#ind = np.arange(0,j_max,1) # plot every time step for saving a movie

# make clean movie directory
!rm -rf {resultsname}/movie
!mkdir {resultsname}/movie

j=0

if len(ind)>1:
# use agg backend for saving pngs
    mpl.use('agg')
else:
    %matplotlib inline

for i in ind:
    print('saved image '+str(j+1)+' out of '+str(len(ind))+' \r',end='')
    
    fig = plt.figure(figsize=(12,12))
            
    plt.suptitle(r'$t=$'+'{:.2f}'.format(365*t[i]/3.154e7)+' d',y=1.04,fontsize=26,bbox=dict(boxstyle='round', facecolor='w', alpha=1))

    plt.subplot(221)
    N_i = N[i]
    storage_i = storage[i]
    p1 = plt.tricontourf(triang,N_i/1e6,levels=np.linspace(0,2,100),cmap='Purples',extend='both')
    plt.tricontour(triang,storage_i,levels=[0.001],colors='k',linestyles='-',linewidths=2)
    # plt.triplot(triang.x, triang.y, triang.triangles,linewidth=1,color='k',alpha=0.1)
    plt.gca().set_aspect('equal', 'box')
    plt.xlabel(r'$x$ (km)',fontsize=20)
    plt.ylabel(r'$y$ (km)',fontsize=20)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    cbar_ax = fig.add_axes([0.13, 0.9, 0.35, 0.02])
    cbar = plt.colorbar(p1,orientation='horizontal',ticks=np.linspace(0,2,6),cax=cbar_ax)
    cbar.set_label(r'$N$ (MPa)',fontsize=20,labelpad=15)
    cbar.ax.tick_params(labelsize=16)
    cbar.ax.xaxis.set_ticks_position('top')
    cbar.ax.xaxis.set_label_position('top')

    
    plt.subplot(222)
    q_i = q_m[i]
    p2 = plt.tricontourf(triang,q_i,cmap='Blues',levels=np.linspace(0,1,100),extend='both')
    plt.tricontour(triang,storage_i,levels=[0.001],colors='k',linestyles='-',linewidths=2)
    plt.gca().set_aspect('equal', 'box')
    plt.xlabel(r'$x$ (km)',fontsize=20)
    plt.gca().yaxis.set_label_position("right")
    plt.gca().yaxis.tick_right()
    plt.ylabel(r'$y$ (km)',fontsize=20)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    cbar_ax = fig.add_axes([0.55, 0.9, 0.35, 0.02])
    cbar = plt.colorbar(p2,orientation='horizontal',ticks=np.linspace(0,1,6),cax=cbar_ax)
    cbar.set_label(r'$|\mathbf{q}|$ (m$^2/$s)',fontsize=20,labelpad=15)
    cbar.ax.tick_params(labelsize=16)
    cbar.ax.xaxis.set_ticks_position('top')
    cbar.ax.xaxis.set_label_position('top')
    
    plot = plt.subplot(212)
    ax2 = plt.gca()
    plt.plot(t[0:i]*365./3.154e7,N_mean[0:i]/1e6,color='purple',linestyle='-',linewidth=3,zorder=100)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.gca().tick_params(axis='y', colors="purple")
    plt.ylabel(r'$\overline{N}$ (MPa)',fontsize=20,color='purple')
    plt.ylim(np.floor(N_mean.max()/1e6),np.floor(N_mean.min()/1e6))
    plt.xlabel(r'$t$ (d)',fontsize=20)
    plt.xlim(0,t[j_max]*365./3.154e7)

    plt.gca().twinx()
    plt.plot(t[0:i]*365./3.154e7,q_max[0:i],color='royalblue',linewidth=3,zorder=10)
    plt.gca().yaxis.set_label_position("right")
    plt.gca().yaxis.tick_right()   
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.gca().tick_params(axis='y', colors="royalblue")
    plt.ylabel(r'$|\mathbf{q}|_\mathrm{max}$ (m$^2$/s) ',fontsize=20,color='royalblue')
    plt.xlabel(r'$t$ (d)',fontsize=20)
    plt.xlim(0,t[j_max]*365./3.154e7)
    plt.ylim(0,np.ceil(q_max.max())+1)

    plot.set_zorder(ax2.get_zorder()+1)
    plot.set_frame_on(False)
    plt.savefig('{}/movie/movie_'.format(resultsname)+str(j),bbox_inches='tight')
    if len(ind)==1:
        plt.show()
    plt.close()
    j+=1